In [1]:

!pip install accelerate==0.26.0
!pip install datasets
!pip install torch
!pip install evaluate
import torch
import evaluate
import sklearn.metrics
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, AutoConfig
from transformers import AutoModelForCausalLM
from transformers import LlamaTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import Trainer
import numpy as np
import pandas as pd
import datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 5.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/huggingface_cli.py", line 49, in main
    servic

First read the file.

In [4]:
data = load_dataset("pauri32/fiqa-2018")
goldstandard_validation=[int(round(data["validation"]["sentiment_score"][i],0)+1) for i in range(len(data["validation"]["sentiment_score"]))]
goldstandard_test=[int(round(data["test"]["sentiment_score"][i],0)+1) for i in range(len(data["test"]["sentiment_score"]))]
goldstandard=goldstandard_validation+goldstandard_test
with open("/content/drive/MyDrive/ipynb/final_project/goldlabel_FiQA.txt", 'w', encoding='utf-8') as f:
  for i in range(len(goldstandard)):
    f.write(str(goldstandard[i]))
    f.write('\n')
sentence_validation=data["validation"]["sentence"]
sentence_test=data["test"]["sentence"]
sentence=sentence_validation+sentence_test
test = datasets.Dataset.from_dict({"text":sentence,"label":goldstandard})
sentiment_score=[int(round(data["train"]["sentiment_score"][i],0)+1) for i in range(len(data["train"]["sentiment_score"]))]
sentence = data["train"]["sentence"]
train = datasets.Dataset.from_dict({"text":sentence,"label":sentiment_score})
dataset = datasets.DatasetDict({"train":train,"validation":test})


Tokenize the text input.

In [5]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
config = AutoConfig.from_pretrained("ProsusAI/finbert")
config.num_labels = 3  # Set the number of classes
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert", config=config)
def tokenize_function(dataset):
  return tokenizer(dataset["text"], padding="max_length", truncation=True)
tokenized_training = dataset["train"].map(tokenize_function, batched=True)
tokenized_testing = dataset["validation"].map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Map:   0%|          | 0/961 [00:00<?, ? examples/s]

Map:   0%|          | 0/252 [00:00<?, ? examples/s]

Set up the pre-trained model.

In [6]:
!PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [7]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    # create Trainer
training_args = TrainingArguments(
  output_dir="/content/drive/MyDrive/ipynb/final_project/finBERT_FiQA_checkpoint.txt",
  learning_rate=2e-5,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=16,
  num_train_epochs=3,
  weight_decay=0.01,
  evaluation_strategy="epoch",
  save_strategy="epoch",
  load_best_model_at_end=True,
)
def compute_metrics(eval_pred):

    f1_metric = evaluate.load("f1")

    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    results = {}
    results.update(f1_metric.compute(predictions=predictions, references = labels, average="micro"))

    return results
trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=tokenized_training,
  eval_dataset=tokenized_testing,
  tokenizer=tokenizer,
  data_collator=data_collator,
  compute_metrics=compute_metrics
)


Train the model.

In [8]:
!PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [9]:
trainer.train()
model.save_pretrained("/content/drive/MyDrive/ipynb/final_project/finBERT_FiQA")

Epoch,Training Loss,Validation Loss,F1
1,No log,0.490647,0.861111
2,No log,0.433259,0.865079
3,No log,0.407735,0.849206


The downstream work can start here to save time. Use the model to produce predictions

In [10]:
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/ipynb/final_project/finBERT_FiQA")
predictions = trainer.predict(tokenized_testing)
preds = np.argmax(predictions.predictions, axis=1)


Save the labels for the downstream work.

In [11]:
pre=preds.tolist()
with open("/content/drive/MyDrive/ipynb/final_project/finBERT_FiQA.txt", 'w', encoding='utf-8') as f:
  for i in range(len(pre)):
    f.write(str(pre[i]))
    f.write('\n')


The downstream work can start here to save time. Find the total accuracy.

In [12]:
with open("/content/drive/MyDrive/ipynb/final_project/finBERT_FiQA.txt", 'r', encoding='utf-8') as f:
    result=(f.read())
result=[result]
result=result[0].split("\n")
label=[]
for i in range(len(result)-1):
  label.append(int(result[i]))
with open("/content/drive/MyDrive/ipynb/final_project/goldlabel_FiQA.txt", 'r', encoding='utf-8') as f:
    result=(f.read())
result=[result]
result=result[0].split("\n")
goldstandard=[]
for i in range(len(result)-1):
  goldstandard.append(int(result[i]))
correct=0
for i in range(len(label)):
  if label[i]==goldstandard[i]:
    correct+=1
accuracy = correct / len(label)
print("Accuracy:", accuracy)

Accuracy: 0.8492063492063492


Find the Precise.

In [14]:
print("Macro Precision:", sklearn.metrics.precision_score(goldstandard, label, average='macro'))
print("Micro Precision:", sklearn.metrics.precision_score(goldstandard, label, average='micro'))

Macro Precision: 0.727668845315904
Micro Precision: 0.8492063492063492


Find the Recall

In [15]:
print("Macro Recall:", sklearn.metrics.recall_score(goldstandard, label, average='macro'))
print("Micro Recall:", sklearn.metrics.recall_score(goldstandard, label, average='micro'))

Macro Recall: 0.43606618274549963
Micro Recall: 0.8492063492063492


Find the F1

In [16]:
print("Macro F1:", sklearn.metrics.f1_score(goldstandard, label, average='macro'))
print("Micro F1:", sklearn.metrics.f1_score(goldstandard, label, average='micro'))

Macro F1: 0.44059182781526657
Micro F1: 0.8492063492063493
